In [27]:
#Testig the ping and installing libraries 
import os
import subprocess
hostname = 'qa-datagov01.nexjqa.local' 
response = os.system("ping -c 1 " + hostname)

#and then check the response...
if response == 0:
  print ('is up!')
else:
  print ('is down!')


print(subprocess.Popen("odbcinst -j", shell=True,stdout=subprocess.PIPE).communicate()[0].decode('utf-8').strip())

# proc = subprocess.Popen(["odbcinst -j"], stdout=subprocess.PIPE, shell=True)
# (out, err) = proc.communicate()
# print ("program output:\n"+ str(out))

is up!
unixODBC 2.3.1
DRIVERS............: /etc/odbcinst.ini
SYSTEM DATA SOURCES: /etc/odbc.ini
FILE DATA SOURCES..: /etc/ODBCDataSources
USER DATA SOURCES..: /home/farhad.rahbarnia/.odbc.ini
SQLULEN Size.......: 8
SQLLEN Size........: 8
SQLSETPOSIROW Size.: 8


In [29]:
'''Connecting to DB'''
import pyodbc
from datetime import datetime
Server='qa-datagov01.nexjqa.local'
DB='RPO_MAJIK_DISCOVERY'
Ver='sql2012'
user='nexj'
password='nexj'
print ("DB CONNECT ATTEMPT")
try:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+Server+';DATABASE='+DB+';UID='+user+';PWD='+password)
    
    print ("SUCCESS")
except Exception as e:
    print ("Error: " + str(e))

# # Create a cursor from the connection
# cursor = cnxn.cursor()

DB CONNECT ATTEMPT
Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")


In [30]:
host = "qa-datagov01.nexjqa.local"
database = "RPO_MAJIK_DISCOVERY"
username='nexj'
password='nexj'
print ("DB CONNECT ATTEMPT")
try:
    cs = 'DSN=%s;UID=%s;PWD=%s;DATABASE=%s;' % (host, username, password, database)
    cnxn = pyodbc.connect(cs)
    print ("SUCCESS")
except Exception as e:
    print ("Error: " + str(e))

DB CONNECT ATTEMPT
Error: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found, and no default driver specified (0) (SQLDriverConnect)')


In [ ]:
#TODO:  NEEDS TO CHANGE 

#first we need to read all of the error codes
sqlStr='SELECT [id], [type]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[errorcode]'
cursor.execute(sqlStr)    
buffers = cursor.fetchall()
# print (buffers)
errorcodes=[]
for buffer in buffers: 
    errorcodes.append ((buffer[0],buffer[1]))  #TODO : make sure buffer is right
#print (errorcodes)  
# Read the equipment 
#
sqlStr='SELECT [id]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[equipment]'  
cursor.execute(sqlStr)    
buffers = cursor.fetchall()
equipments=[]
# eqStr="("

for buffer in buffers: 
    equipments.append (buffer[0]) 
#     eqStr+='[E'+buffer[0].replace("-", "0")+']'# becuase thats how we name the columns in SQL 
#     eqStr+=','


# eqStr+="[errorID])"
# #print (eqStr)
eqStr="([ErrorId],[line],[ErrorType],[DeviceId],[MTTF],[MeanDownT])"


In [ ]:
'''Calculating the mean time to failure(MTTF) '''
from datetime import datetime

c=0

for error in errorcodes:
    c+=1
#     if c==5:
#         break
    
#     k=0
    for equipment in equipments:
        
        sqlStr=' select * FROM [RPO_MAJIK_DISCOVERY].[dbo].[error]  WHERE LTRIM(RTRIM([equipmentId]))=LTRIM(RTRIM(\''+equipment+'\')) AND LTRIM(RTRIM([errorcodeId]))=LTRIM(RTRIM(\''+error[0]+'\')) order by start_time'
#         print (sqlStr)
        cursor.execute(sqlStr)    
        '''Read all at once '''
        rows = cursor.fetchall()
        prev=0  # previouse value
        avgMTTF=0
        avgDown=0
        count=0
        line=1
        strWrite="(" 
        for row in rows:
            if prev==0:
                prev =datetime.strptime(row[3][:16], '%Y-%m-%d %H:%M')
                
            startTime=datetime.strptime(row[3][:16], '%Y-%m-%d %H:%M')
            
            count+=1
            
            avgMTTF+=(startTime-prev).total_seconds()/3600   
            avgDown+=(datetime.strptime(row[4][:16], '%Y-%m-%d %H:%M')-datetime.strptime(row[3][:16], '%Y-%m-%d %H:%M')).total_seconds()/3600 
            prev=datetime.strptime(row[4][:16], '%Y-%m-%d %H:%M')

        if(count==0):
            count=1
        avgMTTF=avgMTTF/count
        avgDown=avgDown/count
        #ErrorID, Line=1, ErrorType, DeviceID and MTF as well as the MeanTotalDowntime
        strWrite+="\'"+error[0]+"\'"+','
        strWrite+=str(line)+','
#         strWrite+="\'"+str(line)+"\'"+','
        strWrite+="\'"+error[1]+"\'"+','
        strWrite+="\'"+equipment+"\'"+','
        strWrite+=str(avgMTTF)+','
        strWrite+=str(avgDown)
#         strWrite+="\'"+str(avgMTTF)+"\'"+','
#         strWrite+="\'"+str(avgDown)+"\'"
        
        strWrite+=")"
        
        sqlStr='INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[MTTFtbl]'+eqStr+' VALUES'+strWrite+';'
#         print (sqlStr)
        cursor.execute(sqlStr)   
        cnxn.commit()
 
    strWrite=""
    print(c)

           
            

    #strWrite+="\'"+error+"\'"+")"
        



#TODO write to SQL and update the table 
#CHANGE THE TABLE NAME AND CONDITIONS 


print("Done")




